This notebook explores a few different potential models of the Milky Way

In [ ]:
using CairoMakie
using Revise
using LilGuys, Arya

In [ ]:
using PythonCall
agama = pyimport("agama")

#agama.setUnits(mass=1e10, length=1, time=T2GYR, G=1)
agama.G

In [ ]:
using Printf

In [ ]:
include("agama_utils.jl")

In [ ]:
function print_agama_units()
    units = pyconvert(Dict, agama.getUnits())
    @printf "G       \t %0.4f\n" pyconvert(Float64, agama.G)
    if "length" ∈ keys(units)
        @printf "length  \t %0.4f kpc\n" units["length"]
        @printf "mass    \t %0.0f M⊙\n" units["mass"]
        @printf "velocity\t %0.2f km/s\n" units["velocity"]
        @printf "time    \t %0.1f Myr\n" units["time"]
    end
end

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"

In [ ]:
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)

In [ ]:
gc = LilGuys.transform(Galactocentric, icrs)

In [ ]:
ep20 = load_agama_potential("EP2020.ini")


In [ ]:
agama.setUnits(length=1,  mass=M2MSUN, time=T2GYR*1e3)
pot = load_agama_potential("EP2020.ini")


print_agama_units()

In [ ]:
orbit = calc_orbit(gc, ep20, units=:code, time=-10)

In [ ]:
plot_r_t(orbit)

In [ ]:
plot_y_z(orbit)

In [ ]:
plot_v_circ(ep20)

In [ ]:
calc_v_circ_pot(vasiliev21, [1,2,3], vasiliev_units=true) * V2KMS

In [ ]:
orbit = calc_orbit(gc, pot)

In [ ]:
plot_y_z(orbit)

In [ ]:
plot_r_t(orbit)

In [ ]:
mm11 = load_agama_potential("mcmillan11.ini")
mm17 = load_agama_potential("mcmillan17.ini")
bt08 = load_agama_potential("BT08.ini")
piffl14 = load_agama_potential("piffl_14.ini")
J95 = load_agama_potential("J95.ini")
galpy14 = load_agama_potential("galpy_2014.ini")

In [ ]:
potentials_light = [
    "EP20" => ep20,
    "bt08" => bt08,
    "galpy14" => galpy14,
    ]


In [ ]:
potentials_heavy = [
    "EP20" => ep20,
    "mcmillan 11" => mm11,
    "piffl14" => piffl14,
    "J95" => J95,

]

In [ ]:
potentials = [
    "EP20" => ep20,
    "mcmillan 17" => mm17,

]

# Potential comparisons

In [ ]:
function plot_all(gc, pot)
    orbit = calc_orbit(gc, pot)
    plot_r_t(orbit) |> display
    plot_y_z(orbit) |> display
end

In [ ]:
function plot_all(gc, pots)

    
    orbits = [calc_orbit(gc, pot) for (name, pot) in pots]

    fig = Figure()
    ax = axis_r_t(fig[1, 1])
    
    for i in 1:length(pots)
        plot_r_t!(ax, orbits[i], label=pots[i].first)
    end

    axislegend()
    
    display(fig)


    fig = Figure()
    ax = axis_y_z(fig[1,1])


    for i in 1:length(pots)
        plot_y_z!(ax, orbits[i], label=pots[i].first)
    end

    axislegend()
    display(fig)
end

In [ ]:
plot_all(gc, potentials)

In [ ]:
plot_all(gc, potentials_light)

In [ ]:
plot_all(gc, potentials_heavy)

# Old Cord

In [ ]:
plot_r_t(orbit_v)

In [ ]:
Mp = 2.32e7*u.Msun
pouliasis17_bulge = Plummer(amp=460*Mp, b=0.3*u.kpc)
pouliasis17_thin = MN75(amp=1700*Mp, a=5.3*u.kpc, b=0.25*u.kpc)
pouliasis17_thick = MN75(amp=1700*Mp, a=2.6*u.kpc, b=0.8*u.kpc)
ah = 14*u.kpc
Mh = 6000*Mp
pouliasis17_halo = 2*NFW(amp=Mh, a=ah) + gp.TwoPowerSphericalPotential(alpha=0, beta=2, amp=Mh, a=ah)

pouliasis17 = pouliasis17_bulge + pouliasis17_thin + pouliasis17_thick + pouliasis17_halo

In [ ]:
pouliasis17b_thin = MN75(amp=1600*Mp, a=4.8*u.kpc, b=0.25*u.kpc)
pouliasis17b_thick = MN75(amp=1700*Mp, a=2.0*u.kpc, b=0.8*u.kpc)
ah = 14*u.kpc
Mh = 9000*Mp
pouliasis17b_halo = 2*NFW(amp=Mh, a=ah) + gp.TwoPowerSphericalPotential(alpha=0, beta=2, amp=Mh, a=ah)

pouliasis17b = pouliasis17b_thin + pouliasis17b_thick + pouliasis17b_halo

builds on pouliasis but with NFW halo and more recent (cephied) data. Very close to galpy.

In [ ]:
R200_NFW(M200) / c

In [ ]:
M200 = 66.3*M0
c = 12.36
Ms = Ms_NFW( M200, c)

Rs = 14.45*u.kpc
Ms

In [ ]:
find_M_c(Ms, Rs)

In [ ]:
M200 = 6.749*M0
c = 12.7988
Ms = Ms_NFW(M200, c)

In [ ]:
Ms

In [ ]:
ablimit_c = {
    "halo": NFW(amp=Ms, a=Rs),
    "disk": pouliasis17_thick + pouliasis17_thin,
    "bulge": pouliasis17_bulge,
}

ablimit20a = [val for key, val in ablimit_c.items()]




In [ ]:
M200 = 82.2*M0
c = 13.04
Ms = Ms_NFW(M200, c)
ablimit_b_comp = {
    "halo": NFW(amp=Ms, a=14.71*u.kpc),
    "disk": pouliasis17_thick + pouliasis17_thin,
    "bulge": pouliasis17_bulge,
}

ablimit20b = [val for key, val in ablimit_b_comp.items()]


In [ ]:
R200_NFW(M200) / c

In [ ]:
plot_Vc(ablimit_c["halo"])
plot_Vc(galpy14_halo)

In [ ]:
plot_Vc(ablimit_b_comp["halo"])
plot_Vc(1.2*galpy14_halo)

In [ ]:
4*np.pi * M0 / (207.0*u.kpc)**3

In [ ]:
shen22_halo = gp.PowerSphericalPotential(amp=0.2275*M0, r1=1*u.kpc, alpha=2+0.43)


c=12
R200=207*u.kpc
Ms = Ms_NFW(105*M0, c) # is 95 in paper but not described well...
shen22_halo_b = NFW(amp=Ms, a=R200/c)

In [ ]:
conv.mass_in_1010msol(vo=Vgp, ro=Rgp)

In [ ]:
shen22_halo.mass(250*u.kpc) / M0

In [ ]:
shen22_halo_b.mass(R200) 

In [ ]:
shen22_halo.mass(10*u.kpc) / M0

In [ ]:
shen22_halo.mass(100*u.kpc) / M0

In [ ]:
shen22_halo.mass(250*u.kpc) / M0

In [ ]:
Rs = np.linspace(10, 250, 1000) * u.kpc

In [ ]:
plt.plot(Rs, [shen22_halo.mass(R) / M0 for R in Rs])
plt.plot(Rs, [shen22_halo_b.mass(R) / M0 for R in Rs])

In [ ]:
plot_Vc(shen22_halo, R_min=10)
plot_Vc(galpy14)
plot_Vc(shen22_halo_b, R_min=10)

In [ ]:
nitschai_halo = gp.TwoPowerTriaxialPotential(amp=41.1*M0, alpha=1.53, beta=3, a=16.8*u.kpc, c=1.14)
nitschai_halo_b = gp.TwoPowerSphericalPotential(amp=44.1*M0, alpha=1.53, beta=3, a=16.8*u.kpc)

In [ ]:
Mh = nitschai_halo.mass(8.2*u.kpc)

In [ ]:
nitschai_halo.dens(8.2*u.kpc, 0*u.kpc)

In [ ]:
Mh / (Mh + mcmillan2017_halo.mass(8.2*u.kpc) + mcmillan2017_thick.mass(8.2*u.kpc) +  mcmillan2017_thin.mass(8.2*u.kpc))

In [ ]:
plot_Vc(nitschai_halo)
plot_Vc(nitschai_halo_b)

plot_Vc(EP2020_halo)

In [ ]:
nitschai = nitschai_halo + mcmillan2017_bulge + mcmillan2017_thick + mcmillan2017_thick + mcmillan2017_h1 + mcmillan2017_h2
nitschai_b = nitschai_halo_b + mcmillan2017_bulge + mcmillan2017_thick + mcmillan2017_thick + mcmillan2017_h1 + mcmillan2017_h2

In [ ]:
mcmillan2017_thin.mass(100)

In [ ]:
mcmillan2017_h2b.surfdens(1, 0)

In [ ]:
(mcmillan2017_thick.dens(1, 0) + mcmillan2017_thin.dens(1, 0)) / (0.3*u.Msun/u.Lsun) / (1.12)

In [ ]:
A_disk = 0.056 / 0.2

In [ ]:
plot_Vc(nitschai, R_min=4, R_max=12)
plot_Vc(mcmillan2017, R_min=4, R_max=12)

plt.ylim(200, 250)

In [ ]:
plot_Vc(nitschai)
plot_Vc(mcmillan2017) #only difference is halo


In [ ]:
Ms = 105*M0
r_s = 20.2*u.kpc
heavy_halo = NFW(amp=Ms, a=r_s)
find_M_c(Ms, r_s)

In [ ]:
plot_Vc(2*galpy14_halo)
plot_Vc(heavy_halo)

In [ ]:
Ms = 60*M0
r_s = 20.2*u.kpc
light_halo = NFW(amp=Ms, a=r_s)
find_M_c(Ms, r_s)

In [ ]:
plot_Vc(galpy14_halo)
plot_Vc(light_halo)

In [ ]:
Ms = 130*M0
r_s = 33*u.kpc
me_flat_halo =  NFW(amp=Ms, a=r_s)
me_flat = EP2020_thin + EP2020_thick + EP2020_bulge + me_flat_halo

find_M_c(Ms, r_s)

In [ ]:
plot_Vc(me_flat_halo)
#plot_Vc(EP2020_halo)
plot_Vc(pouliasis17_halo)

In [ ]:
Ms = 45*M0
r_s = 10*u.kpc
me_steep_halo =  NFW(amp=Ms, a=r_s)

me_steep = EP2020_thin + EP2020_thick + EP2020_bulge + me_steep_halo

find_M_c(Ms, r_s)

In [ ]:
plot_Vc(me_steep_halo)
#plot_Vc(EP2020_halo)
plot_Vc(nitschai_halo)

In [ ]:
plot_Vc(EP2020_halo)
plot_Vc(me_flat_halo)
plot_Vc(me_steep_halo)


In [ ]:
plot_Vc(EP2020_halo)
plot_Vc(heavy_halo)
plot_Vc(light_halo)

In [ ]:
plot_Vc(EP2020)
plot_Vc(me_flat)
plot_Vc(me_steep)


In [ ]:
heavy_mw = EP2020_bulge + EP2020_thin + EP2020_thick + heavy_halo
light_mw = EP2020_bulge + EP2020_thin + EP2020_thick + light_halo

In [ ]:
plot_Vc(me_steep)
plot_Vc(nitschai)

In [ ]:
plot_Vc(me_flat)
plot_Vc(pouliasis17)

In [ ]:
plot_Vc(EP2020)
plot_Vc(heavy_mw)
plot_Vc(light_mw)
plot_Vc(me_steep)
plot_Vc(me_flat)

In [ ]:
potentials = {
    "EP20": EP2020,
    #"mcmillan11": mcmillan2011, # mcmillan is very close to EP2020
    #"mcmillan17": mcmillan2017,
    #"mcmillan17b": mcmillan2017_nogas,
    "galpy14": galpy14,
    # "galpy14_heavy": galpy14_heavy,
    # "pouliaris17": pouliasis17,
    # #"pouliaris17b": pouliasis17b,
    # "ablimit": ablimit20a,
    # "flat": me_flat,

}

In [ ]:
plot_Vc(me_flat)
plot_Vc(EP2020)

In [ ]:
plt.figure(figsize=(7, 3))
for label, model in potentials.items():
    plot_Vc(model, label=label, R_max=60)
arya.Legend(loc=-1)
plt.xscale("asinh")

ticks = [0,1, 10]
plt.xticks(ticks, labels=ticks);

mticks = np.arange(0, 1, 0.1).tolist() + np.arange(1, 10, 1).tolist() + np.arange(10, 60, 10).tolist()
plt.xticks(mticks, minor=True);
plt.savefig("literature_vcirc.pdf")

In [ ]:
plot_Vc(EP2020)
plot_Vc(heavy_mw)
plot_Vc(light_mw)
plot_Vc(me_steep)
plot_Vc(me_flat)
plt.xscale("asinh")


In [ ]:
# acceleration at 1.1 kpc above plane
for label, model in potentials.items():
    plot_Kz(model, label=label)
arya.Legend(loc=-1)

## Components

In [ ]:
my_potentials = {
    "fiducial": EP2020,
    "light": light_mw,
    "heavy": heavy_mw,
    "flat": me_flat,
    "steep": me_steep
}

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jr(pot))

In [ ]:
o.jr(galpy14)

In [ ]:
o.jr(galpy14_heavy)

In [ ]:
o.jr(EP2020)

In [ ]:
o.jr(ablimit20a)

In [ ]:
o.jr(mcmillan2017)

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jz(pot))

In [ ]:
o.jz(galpy14)

In [ ]:
o.jz(galpy14_heavy)

In [ ]:
o.jz(EP2020)

In [ ]:
o.jz(mcmillan2017)

In [ ]:
plot_Vc(EP2020_bulge)
plot_Vc(mcmillan2011_bulge)
plot_Vc(galpy14_bulge)
plot_Vc(pouliasis17_bulge)

In [ ]:
plot_Vc(EP2020_thick + EP2020_thick)
plot_Vc(galpy14_disk)
# plot_Vc(galpy14_disk)

plot_Vc(pouliasis17_thick + pouliasis17_thick)
# plot_Vc(pouliasis17b_thick + pouliasis17b_thick)

In [ ]:
plot_Vc(EP2020_halo)
plot_Vc(mcmillan2011_halo)
plot_Vc(mcmillan2017_halo)
plot_Vc(galpy14_halo)
plot_Vc(2*galpy14_halo)

plot_Vc(pouliasis17_halo)
plot_Vc(pouliasis17b_halo)
plot_Vc(ablimit_b_comp["halo"]) # pretty much same as galpy


In [ ]:
dwarf_galaxy_gc.icrs

In [ ]:
o = Orbit(dwarf_galaxy_gc)

In [ ]:
pot = EP2020
ts = np.linspace(0, -5, 10_000) * u.Gyr
o.integrate(ts, pot)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.R(ts), o.z(ts))
ax.set_aspect(1)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.y(ts), o.z(ts))

ax.set_aspect(1)

In [ ]:
plt.plot(ts, o.r(ts))

In [ ]:
pos_gc = []

ts = np.linspace(0, 0.1, 10_000) * u.Gyr
o.integrate(ts, pot)

# galpy uses left-handed frame :/
pos_gc = coord.SkyCoord(
    x=- o.x(ts), y=o.y(ts), z=o.z(ts), 
    v_x = -o.vx(ts), v_y=o.vy(ts), v_z=o.vz(ts),
    frame=gc_frame)

pos_icrs = pos_gc.transform_to("icrs")

In [ ]:
plt.scatter(pos_icrs.ra, pos_icrs.dec, c=ts)
plt.colorbar()

plt.plot(o.ra(ts), o.dec(ts))

plt.scatter(dwarf_galaxy.ra, dwarf_galaxy.dec)


tpm = 3e7*u.yr
plt.plot(sculptor.ra + sculptor.pm_ra_cosdec / np.cos(sculptor.dec) * tpm * np.array([0, 1]),
        sculptor.dec + sculptor.pm_dec * tpm * np.array([0,1]),
         zorder=5, color=arya.COLORS[2]
        )

plt.xlabel("RA")
plt.ylabel("dec")

In [ ]:
sc2 = dwarf_galaxy_gc.transform_to(coord.ICRS)

# LMC Orbit

In [ ]:
lmc_coord = ICRS(ra = 81,
    dec = -69.75, 
    pmra = 1.8,
    pmdec = 0.35,
    radial_velocity = 260,
    distance = 50.
    )

In [ ]:
lmc_gc = LilGuys.transform(LilGuys.Galactocentric, lmc_coord)

In [ ]:
vasiliev21_frozen = load_agama_potential("vasiliev+21/potential_nolmc.ini")


In [ ]:
vasiliev21 = load_agama_potential("vasiliev+21/potential_evolving.ini")


In [ ]:
orbit_lmc_me = calc_orbit(lmc_gc, vasiliev21_frozen, units=:vasiliev, time=LinRange(0, -10, 1000))


In [ ]:
using CSV, DataFrames

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])

filt = lmc_traj.time .<= 0
lmc_traj = lmc_traj[filt, :]
lmc_traj = DataFrame(reverse(eachrow(lmc_traj)))


lmc_orbit = Orbit(time=lmc_traj.time  * V_T2GYR/ T2GYR, 
    position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
    velocity = [lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]' * V_V2KMS / V2KMS
    )



In [ ]:
include("agama_utils.jl")

In [ ]:
scl_orbit = calc_orbit(gc, vasiliev21_frozen, units=:vasiliev, time=-10)

In [ ]:
scl_lmc_orbit = calc_orbit(gc, vasiliev21, units=:vasiliev, time=lmc_traj.time) # need to pass vasiliev time to this class...

In [ ]:
posvel = pyconvert(Matrix{Float64}, ov(lmc_orbit.time))'

In [ ]:
length(lmc_orbit.time)

In [ ]:
LilGuys.Plots.plot_xyz(lmc_orbit.position, scl_lmc_orbit.position, scl_orbit.position, labels=["LMC", "Scl", "Scl (no lmc)"])

In [ ]:
orbits = [
    "scl" => scl_lmc_orbit,
    "scl nolmc" => scl_orbit,
    "lmc" => lmc_orbit,
    "lmc me" => orbit_lmc_me,
    ]

In [ ]:
plot_y_z(orbits)

In [ ]:
plot_r_t(orbits)

In [ ]:
r_scl_lmc = calc_r(lmc_orbit.position, scl_lmc_orbit.position)

In [ ]:
log10(LilGuys.kpc_to_arcmin(LilGuys.calc_break_radius(8.0 / V2KMS, 0.1 / T2GYR), 83.2))

In [ ]:
log10(20)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )

lines!(lmc_orbit.time * T2GYR, r_scl_lmc)

fig

In [ ]:
lmc_orbit.time[argmin(r_scl_lmc)] * T2GYR

In [ ]:
minimum(r_scl_lmc)

In [ ]:
lmc_orbit.time ./ scl_lmc_orbit.time

In [ ]:
plot_v_circ(vasiliev21, vasiliev_units=true)